In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.neural_network import MLPClassifier

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Data

In [3]:
train = pd.read_csv("purchase600-100cls-15k.lrn.csv")
test = pd.read_csv("purchase600-100cls-15k.tes.csv")

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 602 entries, ID to class
dtypes: int64(602)
memory usage: 45.9 MB


In [5]:
train.head()

,ID,0,1,2,3,4,5,6,7,8,...,591,592,593,594,595,596,597,598,599,class
0,0,0,0,0,1,1,0,1,0,0,...,0,1,0,0,0,1,0,0,0,86
1,1,0,0,0,1,1,0,1,0,1,...,0,1,0,0,1,0,0,0,0,81
2,2,0,1,1,1,1,1,1,0,0,...,1,0,1,0,1,0,0,0,0,3
3,3,0,1,0,1,1,1,1,0,0,...,0,1,0,0,1,0,0,0,0,19
4,4,0,0,0,1,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,76


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Columns: 601 entries, ID to 599
dtypes: int64(601)
memory usage: 22.9 MB


In [7]:
test.head()

,ID,0,1,2,3,4,5,6,7,8,...,590,591,592,593,594,595,596,597,598,599
0,10000,0,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10001,0,1,1,1,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,10002,0,1,1,1,1,0,1,1,1,...,0,1,1,0,0,1,0,0,0,0
3,10003,0,1,0,1,1,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,10004,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X_train = train.drop(["ID", "class"], axis=1)
y_train = train["class"]

X_test = test.drop(["ID"], axis=1)

In [9]:
# Appearence of each class
y_train.value_counts().sort_index()

0      94
1      80
2      40
3     237
4      71
     ... 
95     48
96     37
97     45
98    122
99     96
Name: class, Length: 100, dtype: int64

# Model selection

In [10]:
# Grid Search

cv = StratifiedKFold(5, shuffle=True, random_state=1987)
verbose = 3

## Multi-layer Perceptron

In [11]:
parameters = {
    "hidden_layer_sizes":[(1000), (1500), (2000), (2500)],
    "solver":["lbfgs"],
    "activation":["relu"],
    "alpha":[0.001, 0.01]}
model = GridSearchCV(MLPClassifier(), parameters, cv=cv, verbose=verbose, scoring="accuracy")
model.fit(X_train, y_train)

results = pd.DataFrame(model.cv_results_)
results = results[["param_hidden_layer_sizes", "param_alpha", "mean_test_score", "std_test_score"]]
results.sort_values(["mean_test_score"], ascending=False).head(10)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END activation=relu, alpha=0.001, hidden_layer_sizes=1000, solver=lbfgs;, score=0.789 total time=  53.9s
[CV 2/5] END activation=relu, alpha=0.001, hidden_layer_sizes=1000, solver=lbfgs;, score=0.748 total time= 1.1min
[CV 3/5] END activation=relu, alpha=0.001, hidden_layer_sizes=1000, solver=lbfgs;, score=0.793 total time=  59.3s
[CV 4/5] END activation=relu, alpha=0.001, hidden_layer_sizes=1000, solver=lbfgs;, score=0.794 total time= 1.3min
[CV 5/5] END activation=relu, alpha=0.001, hidden_layer_sizes=1000, solver=lbfgs;, score=0.787 total time=  57.3s
[CV 1/5] END activation=relu, alpha=0.001, hidden_layer_sizes=1500, solver=lbfgs;, score=0.785 total time= 1.1min
[CV 2/5] END activation=relu, alpha=0.001, hidden_layer_sizes=1500, solver=lbfgs;, score=0.787 total time= 1.0min
[CV 3/5] END activation=relu, alpha=0.001, hidden_layer_sizes=1500, solver=lbfgs;, score=0.791 total time=  59.8s
[CV 4/5] END activation=relu

,param_hidden_layer_sizes,param_alpha,mean_test_score,std_test_score
3,2500,0.001,0.8113,0.004082
7,2500,0.01,0.8028,0.006129
5,1500,0.01,0.8016,0.005361
6,2000,0.01,0.8007,0.013545
4,1000,0.01,0.7994,0.018717
2,2000,0.001,0.7936,0.011569
1,1500,0.001,0.7895,0.006671
0,1000,0.001,0.7823,0.017360


# Final model

In [12]:
best_model = model.best_estimator_
best_model.fit(X_train, y_train)

In [ ]:
predictions = pd.DataFrame(test["ID"])
predictions["class"] = best_model.predict(X_test)

In [ ]:
predictions.to_csv("submission.csv", index=False)